In [2]:
import des_functions as des
import astropy.io.fits as fits 
import numpy as np
import glob
from more_itertools import locate

In [3]:
data_dir = '/home/b7009348/WGL_project/DES-data/'
im3file = 'y1_im3shape_shapes.fits'
mcalfile = 'y1_metacalibration_shapes.fits'
lensfile = 'DES_Y1A1_3x2pt_redMaGiC_zerr_CATALOG.fits'

### Create matched catalogues

In [8]:
des.cut_flags(im3file, method='im3')

Opening file...
Locating flags...
Flags located, slicing data...
Data sliced, writing to new file...
Runtime: 408.955


In [9]:
des.cut_flags(mcalfile, method='mcal')

Opening file...
Locating flags...
Flags located, slicing data...
Data sliced, writing to new file...
Runtime: 596.622


In [7]:
im3cut = 'y1_im3_flags=0.fits'
mcalcut = 'y1_mcal_flags=0.fits'
zfile = 'mcal_BPZ_(binning).fits'

In [5]:
des.match_catalogues(im3cut, mcalcut, zfile)

Opening files...
Locating NaNs...
Slicing NaNs...
Matching sources...
Finding ID intersections between catalogues...
17865244 sources matched
Slicing im3 data...
Saving im3 data to new file...
Slicing mcal data...
Saving mcal data to new file...
Runtime: 571.635


### Cut lens file to redshift bins

In [16]:
zmins = [0.15, 0.3, 0.45, 0.6, 0.75]
zmaxes = [0.3, 0.45, 0.6, 0.75, 0.9]

for i in range(len(zmins)):
    des.cut_lenses(lensfile, zmin=zmins[i], zmax=zmaxes[i])

Opening files...
Locating lenses in range 0.15 - 0.3...
Lenses in range found, slicing data...
Data sliced, writing to new file...
Runtime: 0.323036
Opening files...
Locating lenses in range 0.3 - 0.45...
Lenses in range found, slicing data...
Data sliced, writing to new file...
Runtime: 0.223156
Opening files...
Locating lenses in range 0.45 - 0.6...
Lenses in range found, slicing data...
Data sliced, writing to new file...
Runtime: 0.254078
Opening files...
Locating lenses in range 0.6 - 0.75...
Lenses in range found, slicing data...
Data sliced, writing to new file...
Runtime: 0.17903
Opening files...
Locating lenses in range 0.75 - 0.9...
Lenses in range found, slicing data...
Data sliced, writing to new file...
Runtime: 0.120048


In [5]:
with fits.open(data_dir+'gold_cut_y1_im3_shapes_matched_(nans).fits') as hdu:
    data = hdu[1].data
    im3_ID = data['coadd_objects_id']
del data

# load in z_mc for all sources
with fits.open(data_dir+'mcal_BPZ_(binning).fits') as hdu:
    data = hdu[1].data
    im3_zmc = data['z_mc']
    zmc_ID = data['coadd_objects_id']
del data

# match z_mc values to source IDs
matches, zmc_indices, shape_indices = np.intersect1d(zmc_ID, im3_ID, return_indices=True)

# slice z_mc values to only to those in matched catalogue
im3_cut = im3_zmc[zmc_indices]

del matches, zmc_indices, shape_indices, zmc_ID, im3_ID

x = np.isnan(im3_cut)
y = []
for i in range(len(x)):
    if x[i] == True:
        y.append(x[i])
    
print(np.size())

TypeError: _size_dispatcher() missing 1 required positional argument: 'a'

In [6]:
x = np.isnan(im3_cut)
if x.any() == True:
    print('yikes')

yikes


In [7]:
print(np.size(im3_cut))

17865244


In [28]:
with fits.open(data_dir+'y1_im3_shapes_matched.fits') as hdu:
    data = hdu[1].data
    im3_ID = data['coadd_objects_id']
del data

# load in z_mc for all sources
with fits.open(data_dir+'mof_BPZ_(stacking).fits') as hdu:
    data = hdu[1].data
    im3_zmc = data['z_mc']
    zmc_ID = data['coadd_objects_id']
del data

# match z_mc values to source IDs
matches, zmc_indices, shape_indices = np.intersect1d(zmc_ID, im3_ID, return_indices=True)

# slice z_mc values to only to those in matched catalogue
im3_zmc = im3_zmc[zmc_indices]

print(im3_zmc[100:200])

del matches, zmc_indices, shape_indices, zmc_ID, im3_ID

[0.57054426 0.68682418 0.54534949 0.58047495 0.3845813  0.77290494
 0.67648208 0.43320593 0.7910586  0.29173101 0.63485931 0.6551489
 0.62785127 0.44386425 0.57018634 0.58110151 0.56906392 0.09365372
 0.42350288 0.38641842 0.28892036 0.73161776 0.59433493 1.01790242
 0.69441394 0.57030637 0.34025371 0.55797283 0.77906071 2.10822018
 0.79504588 0.79689796 0.20804606 0.20086831 0.3613483  0.15532033
 0.59125006 0.18985658 0.42483064 0.50202849 0.47222988 0.75362123
 0.70048697 0.32275801 0.55538035 0.69982425 0.3222205  0.34223948
 0.53289972 0.6282572  0.5780427  0.88159896 0.91829414 0.79065519
 0.55122689 0.51032127 1.85619327 0.16443794 0.73669616 0.44454861
 0.3697183  1.18908265 0.10417111 0.30441146 0.20180837 0.22436716
 0.55639066 0.57957825 0.52521115 0.59577409 0.51491974 0.76630074
 0.12424476 0.24281978 0.86172354 0.73480898 0.45817866 0.43311511
 0.19900989 0.54905832 0.5350536  0.77203373 0.40875619 0.40205463
 0.6619122  0.86279038 0.70184616 0.25846367 0.56989483 0.39491

In [27]:
with fits.open(data_dir+'y1_im3_shapes_matched.fits') as hdu:
    data1 = hdu[1].data
    
print(np.size(data))

with fits.open(data_dir+'y1_mcal_shapes_matched_(nans).fits') as hdu:
    data2 = hdu[1].data
    
print(np.size(data2)-np.size(data1))

17865244
50657


In [8]:
with fits.open(data_dir+zfile) as hdu:
    data = hdu[1].data

In [9]:
print(data.columns)

ColDefs(
    name = 'template_id'; format = 'K'
    name = 'coadd_objects_id'; format = 'K'
    name = 'r11'; format = 'D'
    name = 'r22'; format = 'D'
    name = 'template_type'; format = 'D'
    name = 'median_z'; format = 'D'
    name = 'mean_z'; format = 'D'
    name = 'minchi2'; format = 'D'
    name = 'mode_z'; format = 'D'
    name = 'z_sigma68'; format = 'D'
    name = 'z_sigma'; format = 'D'
    name = 'kl_post_prior'; format = 'D'
    name = 'z_mc'; format = 'D'
)


### Download gold catalogues

In [ ]:
#des.download_gold()

In [5]:
des.cut_gold(filename='y1_im3_shapes_matched_(nans).fits')

Opening files...
Locating gold flags...
Flags located, slicing data...
Saving to new file...
Runtime: 975.306


In [6]:
with fits.open(data_dir+'gold_cut_y1_im3_shapes_matched_(nans).fits') as hdu:
    data1 = hdu[1].data
    
with fits.open(data_dir+'y1_im3_shapes_matched_(nans).fits') as hdu:
    data2 = hdu[1].data
    
print(np.size(data1), np.size(data2))

17865244 17865244
